In [21]:
from flask import Flask,request,jsonify
import joblib
import json
import numpy as np
import base64
import cv2
from tensorflow import keras

# Loads different Functions, Variables and Models which are used when we get request to the server

In [22]:

__class_name_to_number = {}
__class_number_to_name = {}

__model = None
__model_CNN=None

# Function which should be called for loading all the Models
def load_saved_artifacts():
    print("loading saved artifacts...start")
    global __class_name_to_number
    global __class_number_to_name

    with open("./artifacts/class_dict.json", "r") as f:
        __class_name_to_number = json.load(f)
        __class_number_to_name = {v:k for k,v in __class_name_to_number.items()}

    global __model
    global __model_CNN
    if __model is None:
        with open('./artifacts/saved_model.pkl', 'rb') as f:
            __model = joblib.load(f)
    if __model_CNN is None:
        __model_CNN = keras.models.load_model('save_model_CNN')
    print("loading saved artifacts...done")


#Function to perform classifcation 
def classify_image1(image_base64_data, CNN=False,file_path=None):

    imgs = get_cropped_image_if_2_eyes(file_path, image_base64_data)

    result = []
    
    ## If CNN is false perform use Logistic regression to perform classification else
    ## use Convolution Neural Network to perform classification
    if CNN==False:
        for img in imgs:
            scalled_raw_img = cv2.resize(img, (32, 32))
            combined_img = scalled_raw_img.reshape(32 * 32 * 3, 1)

            len_image_array = 32*32*3 

            final = combined_img.reshape(1,len_image_array).astype(float)
            print("logistic",__model.predict_proba(final))
            result.append({
                'class': class_number_to_name(__model.predict(final)[0]),
                'class_probability': np.around(__model.predict_proba(final)*100,2).tolist()[0],
                'class_dictionary': __class_name_to_number
            })
    else:
        for img in imgs:
            scalled_raw_img = cv2.resize(img, (32, 32))
            combined_img = scalled_raw_img.reshape(1,32,32,3,1)
            final = combined_img.reshape(1,32,32,3,1).astype(float)
            ans=__model_CNN.predict(final)[0]
            cls_prob=[]
            print("neural network",ans)
            for i in ans:
                cls_prob.append(round(i*100,2))
            result.append({
                'class': class_number_to_name(np.argmax(ans)),
                'class_probability':cls_prob,
                'class_dictionary': __class_name_to_number
            })
    return result

# Funciton to convert the number to class_name
def class_number_to_name(class_num):
    return __class_number_to_name[class_num]

# Funciton to convert the image in base 64 string format to normal image format which OpenCV can understand
def get_cv2_image_from_base64_string(b64str):
    '''
    credit: https://stackoverflow.com/questions/33754935/read-a-base-64-encoded-image-from-memory-using-opencv-python-library
    :param uri:
    :return:
    '''
    encoded_data = b64str.split(',')[1]
    nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img

# Function returns face of the Image if its two eyes are detected else returns None
def get_cropped_image_if_2_eyes(image_path, image_base64_data):
    face_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_eye.xml')

    if image_path:
        img = cv2.imread(image_path)
    else:
        img = get_cv2_image_from_base64_string(image_base64_data)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    cropped_faces = []
    for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                cropped_faces.append(roi_color)
    return cropped_faces


# Server which takes requests and send responses

In [26]:
app=Flask(__name__)
load_saved_artifacts()
@app.route("/hello")
def hello():
    return 'hi'

@app.route('/classify_image', methods=['POST'])
def classify_image():
    image_data = request.form['image_data']
    data=classify_image1(image_data)
    response = jsonify(data)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

@app.route('/classify_image_cnn',methods=['POST'])
def classify_image_CNN():
    image_data = request.form['image_data']
    data=classify_image1(image_data,CNN=True)
    response = jsonify(data)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

app.run(port=5000)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


loading saved artifacts...start
loading saved artifacts...done


127.0.0.1 - - [24/Mar/2021 15:47:25] "POST /classify_image HTTP/1.1" 200 -


logistic [[0.91958059 0.02980842 0.01684404 0.01037335 0.02339359]]


127.0.0.1 - - [24/Mar/2021 15:47:48] "POST /classify_image_cnn HTTP/1.1" 200 -


...

logistic [[0.08992    0.02123942 0.40801141 0.15349732 0.32733185]]


127.0.0.1 - - [24/Mar/2021 15:52:06] "POST /classify_image HTTP/1.1" 200 -


logistic [[0.02339079 0.07818702 0.06198844 0.39692641 0.43950734]]


127.0.0.1 - - [24/Mar/2021 15:52:32] "POST /classify_image_cnn HTTP/1.1" 200 -


neural network [7.7101014e-08 2.6373511e-05 9.9969065e-01 3.6494450e-06 2.7926906e-04]
